**Before running locally you may want to set up .env file**  
For details look at [README](README.md)

# Data preparation

Convert datasets, calculate additional features and upload to Huggingface.  
For specific dataset configuration look [here](config/datasets_config.py)

In [ ]:
from config.datasets_config import Dataset
from utils.datasets import get

ds = get(Dataset.SLUE_HVB, allow_generation=True, force_regenerate=True)